In [135]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import requests
from io import BytesIO

# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Thai_export_by_product.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    sheet_data = pd.read_excel(BytesIO(response.content), sheet_name='Sheet3')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")

years_columns = sheet_data.columns[2:]  
export_data = sheet_data.iloc[:, 2:].copy() 
export_data.insert(0, 'Product', sheet_data.iloc[:, 0])  
export_data_melted = export_data.melt(id_vars='Product', var_name='Year', value_name='Export Value')
export_data_melted['Year'] = pd.to_numeric(export_data_melted['Year'], errors='coerce')
export_data_melted['Export Value'] = pd.to_numeric(export_data_melted['Export Value'], errors='coerce')

custom_colors = {
    'Bird eggs 3': 'firebrick',
    'Bulk grains 2': 'orange',
    'Cashew nuts': 'gold',
    'Cereal and bakery foods 4/5': 'pink',
    'Fruit juices 4': 'blue',
    'Milled grain products 3': 'green',
    'Prepared fish and shellfish 4': 'purple',
    'Prepared tree nuts 3': 'cyan',
    'Rice and products': 'yellow',
    'Total fruit and preparations 1': 'brown',
    'Total grains and products 1': 'teal',
    'Total nuts 1': 'lightblue',
    'Tree nuts 2': 'darkblue'
}

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Thailand's Export Dashboard", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Select Product Category:"),
        dcc.Dropdown(
            id='product-filter',
            options=[{'label': product, 'value': product} for product in export_data_melted['Product'].unique()],
            value=export_data_melted['Product'].unique(), 
            multi=True  
        )
    ], style={'width': '50%', 'margin': '10px auto'}),
    
    html.Div([
        html.Label("Select Year:"),
        dcc.RangeSlider(
            id='year-filter',
            min=1999,
            max=2023,
            marks={year: str(year) for year in range(1999, 2024, 2)},  
            value=[1999, 2023]  
        )
    ], style={'width': '80%', 'margin': '10px auto'}),
    
    dcc.Graph(id='export-line-chart'),
])

@app.callback(
    Output('export-line-chart', 'figure'),
    [Input('product-filter', 'value'),
     Input('year-filter', 'value')]
)
def update_chart(selected_products, selected_year_range):
    
    filtered_data = export_data_melted[
        (export_data_melted['Product'].isin(selected_products)) &
        (export_data_melted['Year'] >= selected_year_range[0]) &
        (export_data_melted['Year'] <= selected_year_range[1])
    ]
    
   
    fig = px.line(
        filtered_data,
        x='Year',
        y='Export Value',
        color='Product',
        title="Thailand's Exports to the US by Product Category (1999-2023)",
        labels={'Export Value': 'Export Value (in million USD)', 'Year': 'Year'},
        markers=True,
        color_discrete_map=custom_colors  
    )
    fig.update_layout(
        legend_title="Product Categories",
        xaxis=dict(tickmode='linear', dtick=1),  
        template="plotly_white",
        width=1100,
        height=600,
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8061)  


In [137]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
import requests
from io import BytesIO

# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Competitor.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    data = pd.ExcelFile(BytesIO(response.content))
    df = data.parse('Sheet1')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")

# Clean the dataset
df.columns = df.iloc[0]  
df = df[1:]  
df.columns = ['Product Category', 'Country', 'Market Share']  
df['Market Share'] = pd.to_numeric(df['Market Share'], errors='coerce')

custom_colors = {
    'Brazil': 'firebrick',
    'Canada': 'salmon',
    'China': 'orange',
    'Germany': 'gold',
    'India': 'khaki',
    'Indonesia': 'darkkhaki',
    'Italy': 'forestgreen',
    'Mexico': 'darkolivegreen',
    'Netherlands': 'yellowgreen',
    'Pakistan': 'mediumaquamarine',
    'Philippines': 'mediumturquoise',
    'Poland': 'steelblue',
    'Thailand': 'lightskyblue',
    'Turkey': 'slateblue',
    'United Kingdom': 'thistle',
    'Vietnam': 'palevioletred',
    'Other': '#ffffff'  
}

category_order = sorted(df['Country'].unique(), key=lambda x: (x == 'Other', x))

fig = px.bar(
    df,
    x="Product Category",
    y="Market Share",
    color="Country",
    title="Market Share Percentage by Product Category",
    labels={"Market Share": "Market Share (%)"},
    hover_data=["Country", "Market Share"],
    color_discrete_map=custom_colors,  
    category_orders={"Country": category_order}  
)

fig.update_layout(
    xaxis_title="Product Category",
    yaxis_title="Market Share (%)",
    legend_title="Country",
    barmode="stack",  
    template="plotly_white",
    width=1000, 
    height=600,  
    font=dict(size=14),  
)

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Market Share Dashboard", style={'text-align': 'center'}),
    dcc.Graph(figure=fig)  
])

if __name__ == '__main__':
    app.run_server(debug=True, port=8060)  

In [139]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
import requests
from io import BytesIO

# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Value_Infation_USimports.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    data = pd.ExcelFile(BytesIO(response.content))
    df = data.parse('Sheet1')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")


df = pd.read_excel(Sheet_data3, sheet_name=sheet_name)

column_mapping = {
    "Import Food group": "Category",
    "Value (Million dollars)": "Average Food Value (Million USD)",
    "Price Average (Dollars per metric ton)": "Average Price (Dollars/Metric Ton)",
    "Average inflation": "Average Inflation (%)"
}
df = df.rename(columns=column_mapping)

fig = px.scatter(
    df,
    x="Average Food Value (Million USD)",
    y="Average Price (Dollars/Metric Ton)",
    size="Average Inflation (%)",
    color="Category",
    hover_name="Category",
    title="Bubble Chart: Potential Products for Export",
    labels={
        "Average Food Value (Million USD)": "Food Value (Million USD)",
        "Average Price (Dollars/Metric Ton)": "Price (Dollars/Metric Ton)",
    },
    color_discrete_sequence=px.colors.qualitative.Vivid,  
)

fig.update_layout(
    xaxis_title="Average Food Value (Million USD)",
    yaxis_title="Average Price (Dollars/Metric Ton)",
    legend_title="Product Categories",
    template="plotly_white",
    width=1000,  
    height=500,  
)

app = Dash(__name__)

app.layout = html.Div([
    html.H1("US Imports: Value, Price, and Inflation Dashboard", style={'text-align': 'center'}),
    dcc.Graph(figure=fig)  
])

if __name__ == '__main__':
    app.run_server(debug=True, port=8062)  

In [141]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import requests
from io import BytesIO

# ----- Dashboard 1 Data -----
# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Thai_export_by_product.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    sheet_data = pd.read_excel(BytesIO(response.content), sheet_name='Sheet3')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")

years_columns = sheet_data.columns[2:]
export_data = sheet_data.iloc[:, 2:].copy()
export_data.insert(0, 'Product', sheet_data.iloc[:, 0])
export_data_melted = export_data.melt(id_vars='Product', var_name='Year', value_name='Export Value')
export_data_melted['Year'] = pd.to_numeric(export_data_melted['Year'], errors='coerce')
export_data_melted['Export Value'] = pd.to_numeric(export_data_melted['Export Value'], errors='coerce')

custom_colors_1 = {
    'Bird eggs 3': 'firebrick',
    'Bulk grains 2': 'orange',
    'Cashew nuts': 'gold',
    'Cereal and bakery foods 4/5': 'pink',
    'Fruit juices 4': 'blue',
    'Milled grain products 3': 'green',
    'Prepared fish and shellfish 4': 'purple',
    'Prepared tree nuts 3': 'cyan',
    'Rice and products': 'yellow',
    'Total fruit and preparations 1': 'brown',
    'Total grains and products 1': 'teal',
    'Total nuts 1': 'lightblue',
    'Tree nuts 2': 'darkblue'
}

# ----- Dashboard 2 Data -----
# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Competitor.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    data = pd.ExcelFile(BytesIO(response.content))
    df = data.parse('Sheet1')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")
df_market_share = data.parse('Sheet1')
df_market_share.columns = df_market_share.iloc[0]
df_market_share = df_market_share[1:]
df_market_share.columns = ['Product Category', 'Country', 'Market Share']
df_market_share['Market Share'] = pd.to_numeric(df_market_share['Market Share'], errors='coerce')

custom_colors_2 = {
    'Brazil': 'firebrick', 'Canada': 'salmon', 'China': 'orange', 'Germany': 'gold', 'India': 'khaki',
    'Indonesia': 'darkkhaki', 'Italy': 'forestgreen', 'Mexico': 'darkolivegreen', 'Netherlands': 'yellowgreen',
    'Pakistan': 'mediumaquamarine', 'Philippines': 'mediumturquoise', 'Poland': 'steelblue',
    'Thailand': 'lightskyblue', 'Turkey': 'slateblue', 'United Kingdom': 'thistle', 'Vietnam': 'palevioletred',
    'Other': '#ffffff'
}
category_order = sorted(df_market_share['Country'].unique(), key=lambda x: (x == 'Other', x))

# ----- Dashboard 3 Data -----
# Raw URL
url = "https://github.com/Sakuljai/BEMM461-Final_Visualisation-Project/raw/refs/heads/main/Value_Infation_USimports.xlsx"

# Fetch the file content
response = requests.get(url)
if response.status_code == 200:
    data = pd.ExcelFile(BytesIO(response.content))
    df = data.parse('Sheet1')
else:
    raise Exception(f"Failed to fetch the file. Status code: {response.status_code}")

df_bubble = pd.read_excel(Sheet_data3, sheet_name=sheet_name3)
column_mapping = {
    "Import Food group": "Category",
    "Value (Million dollars)": "Average Food Value (Million USD)",
    "Price Average (Dollars per metric ton)": "Average Price (Dollars/Metric Ton)",
    "Average inflation": "Average Inflation (%)"
}
df_bubble = df_bubble.rename(columns=column_mapping)

# ----- App Initialization -----
app = dash.Dash(__name__)

# ----- App Layout -----
app.layout = html.Div([
    html.H1("Combined Dashboard for Thailand's Export Analysis", style={'text-align': 'center'}),
    
    dcc.Tabs([
        # Tab 1: Line Chart
        dcc.Tab(label="Exports Over Time", children=[
            html.Div([
                html.H3("Thailand's Exports to the US", style={'text-align': 'center'}),
                
                html.Div([
                    html.Label("Select Product Category:"),
                    dcc.Dropdown(
                        id='product-filter',
                        options=[{'label': product, 'value': product} for product in export_data_melted['Product'].unique()],
                        value=export_data_melted['Product'].unique(),
                        multi=True
                    )
                ], style={'width': '50%', 'margin': '10px auto'}),
                
                html.Div([
                    html.Label("Select Year:"),
                    dcc.RangeSlider(
                        id='year-filter',
                        min=1999, max=2023,
                        marks={year: str(year) for year in range(1999, 2024, 2)},
                        value=[1999, 2023]
                    )
                ], style={'width': '80%', 'margin': '10px auto'}),
                
                dcc.Graph(id='export-line-chart')
            ])
        ]),
        
        # Tab 2: Market Share
        dcc.Tab(label="Market Share Analysis", children=[
            html.Div([
                html.H3("Market Share Percentage by Product Category", style={'text-align': 'center'}),
                dcc.Graph(
                    figure=px.bar(
                        df_market_share,
                        x="Product Category", y="Market Share", color="Country",
                        title="Market Share Percentage by Product Category",
                        color_discrete_map=custom_colors_2,
                        category_orders={"Country": category_order},
                        barmode="stack",
                        labels={"Market Share": "Market Share (%)"}
                    ).update_layout(width=1000, height=600)
                )
            ])
        ]),
        
        # Tab 3: Bubble Chart
        dcc.Tab(label="Potential Products Analysis", children=[
            html.Div([
                html.H3("US Imports: Value, Price, and Inflation", style={'text-align': 'center'}),
                dcc.Graph(
                    figure=px.scatter(
                        df_bubble,
                        x="Average Food Value (Million USD)",
                        y="Average Price (Dollars/Metric Ton)",
                        size="Average Inflation (%)",
                        color="Category",
                        hover_name="Category",
                        title="Bubble Chart: Potential Products for Export",
                        color_discrete_sequence=px.colors.qualitative.Vivid
                    ).update_layout(width=1000, height=500)
                )
            ])
        ])
    ])
])

# ----- Callbacks -----
@app.callback(
    Output('export-line-chart', 'figure'),
    [Input('product-filter', 'value'), Input('year-filter', 'value')]
)
def update_chart(selected_products, selected_year_range):
    filtered_data = export_data_melted[
        (export_data_melted['Product'].isin(selected_products)) &
        (export_data_melted['Year'] >= selected_year_range[0]) &
        (export_data_melted['Year'] <= selected_year_range[1])
    ]
    fig = px.line(
        filtered_data, x='Year', y='Export Value', color='Product',
        labels={'Export Value': 'Export Value (in million USD)', 'Year': 'Year'},
        color_discrete_map=custom_colors_1, markers=True
    )
    return fig

# ----- Run the App -----
if __name__ == '__main__':
    app.run_server(debug=True, port=8063)
